In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly_express as px
import pandas_profiling

#lines below let allow multiple results from a line of code to be shown e.g. df.head() + df.columns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#this allows us to see all of our columns or rows in jupyter notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#filter future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#set our code up so that it doesn't display scientific notation, we want full numbers
pd.set_option('display.float_format', '{:.2f}'.format)

#using dask for large data
import dask.dataframe as dd
from dask.distributed import Client

In [65]:
#read in all of the data, had to read in with dask since this is too large for my RAM
building_metadata = pd.read_csv('building_metadata.csv')
#test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv', chunksize = 3000000)
#weather_test = pd.read_csv('weather_test.csv')
weather_train = pd.read_csv('weather_train.csv', chunksize = 3000000)

## Working with BIG DATA using chunksize
- since the data is millions of rows, I can only read it in using chunks and manipulate it using chunks

In [66]:
#just drop all NA values to create a training set for baseline
chunk_list = []
for chunk in train:
    chunk = chunk.dropna()
    chunk = pd.merge(chunk, building_metadata, on = 'building_id')
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)
#print(f'The size of our merged training set is {df_concat.shape}')

#get our weather chunks
weather_list = []
for chunk in weather_train:
    chunk = pd.merge(chunk, df_concat)
    weather_list.append(chunk)
weather_concat = pd.concat(weather_list)
print(f'The size of our total train set is: {weather_concat.shape}')

The size of our total train set is: (20125605, 16)


In [67]:
#now lets take a look a the head of our data
print('Head of training data')
weather_concat.head()
print(' ')
print('Tail of training data')
weather_concat.tail()

Head of training data


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,building_id,meter,meter_reading,primary_use,square_feet,year_built,floor_count
0,0,2016-01-01 00:00:00,25.00,6.00,20.00,nan,1019.70,0.00,0.00,0,0,0.00,Education,7432,2008.00,nan
1,0,2016-01-01 00:00:00,25.00,6.00,20.00,nan,1019.70,0.00,0.00,1,0,0.00,Education,2720,2004.00,nan
2,0,2016-01-01 00:00:00,25.00,6.00,20.00,nan,1019.70,0.00,0.00,2,0,0.00,Education,5376,1991.00,nan
3,0,2016-01-01 00:00:00,25.00,6.00,20.00,nan,1019.70,0.00,0.00,3,0,0.00,Education,23685,2002.00,nan
4,0,2016-01-01 00:00:00,25.00,6.00,20.00,nan,1019.70,0.00,0.00,4,0,0.00,Education,116607,1975.00,nan


 
Tail of training data


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,building_id,meter,meter_reading,primary_use,square_feet,year_built,floor_count
20125600,15,2016-12-31 23:00:00,1.70,nan,-5.60,-1.00,1008.50,180.00,8.80,1444,0,8.75,Entertainment/public assembly,19619,1914.00,nan
20125601,15,2016-12-31 23:00:00,1.70,nan,-5.60,-1.00,1008.50,180.00,8.80,1445,0,4.83,Education,4298,nan,nan
20125602,15,2016-12-31 23:00:00,1.70,nan,-5.60,-1.00,1008.50,180.00,8.80,1446,0,0.00,Entertainment/public assembly,11265,1997.00,nan
20125603,15,2016-12-31 23:00:00,1.70,nan,-5.60,-1.00,1008.50,180.00,8.80,1447,0,159.57,Lodging/residential,29775,2001.00,nan
20125604,15,2016-12-31 23:00:00,1.70,nan,-5.60,-1.00,1008.50,180.00,8.80,1448,0,2.85,Office,92271,2001.00,nan


In [70]:
#what does our missing data look like?
weather_concat.isna().sum()

site_id                      0
timestamp                    0
air_temperature           6163
cloud_coverage         8734870
dew_temperature           9645
precip_depth_1_hr      3658528
sea_level_pressure     1141174
wind_direction         1358553
wind_speed               53181
building_id                  0
meter                        0
meter_reading                0
primary_use                  0
square_feet                  0
year_built            12113306
floor_count           16630052
dtype: int64